# Construyamos un App con Gen AI ✨

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/EnriqueVilchezL/ai_workshop_2025_gen_ai_session_2/blob/main/complete.ipynb)

En este taller vamos a aprender a utilizar grandes modelos de lenguaje de manera local, y a construir una pequeña aplicación encima de ese servicio.

## 1. Instalar dependencias

### 1.1. Ollama

Primero, vamos a instalar **Ollama** en la máquina que nos ofrece Google Colab.

**Ollama** es una herramienta de código abierto que permite ejecutar modelos de lenguaje de gran tamaño (LLM) directamente en sus computadoras, sin necesidad de conexión a Internet ni servicios en la nube. Esto la convierte en una opción ideal para desarrolladores, investigadores y empresas que buscan mantener el control total sobre sus datos y reducir costos asociados a APIs externas.

Para más información pueden visitar [Ollama](https://ollama.com/).

En un sistema Linux, Ollama se puede instalar con:

In [ ]:
!sudo apt -q update
!sudo apt -q install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (218 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
41 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u

### 1.2. Python

Ahora, vamos a instalar los paquetes necesarios para ejecutar nuestra aplicación en python. Las principales bibliotecas que vamos a usar son:

1. Langchain: Permite usar LLMs de manera sencilla, ya sea de servicios en la nube o local. Para más información pueden visitar [Langchain](https://www.langchain.com/).

2. Gradio: Permite montar interfaces web para modelos de inteligencia artificial de forma sencilla. Para más información pueden visitar [Gradio](https://www.gradio.app/).

In [ ]:
%pip install langchain_community gradio ollama pypdf

 ## 2. Iniciando con LLMs locales

 Ahora, vamos a utilizar LLMs de forma local, descargando modelos en la máquina de Google Colab y aprovechando la GPU gratuita que nos ofrece.

### 2.1. Correr ollama

Primero, vamos a correr ollama. Ollama funciona como un servidor que escucha en un puerto de su computadora local, en el cual va a ejecutar LLMs. Normalmente, ollama se corre desde una terminal con el comando "ollama serve", pero como no tenemos acceso a una terminal para la máquina de Google Colab, vamos a utilizar la biblioteca de [Ollama en python](https://github.com/ollama/ollama-python).

In [ ]:
import threading
import subprocess
import time

def run_ollama_server() -> None:
    subprocess.Popen(["ollama", "serve"])

threading.Thread(target=run_ollama_server).start()
time.sleep(5)

Ahora, vamos a descargar un modelo de la nube a la máquina local.

In [ ]:
import ollama

ollama.pull("llama3.1:8b")

ProgressResponse(status='success', completed=None, total=None, digest=None)

Podemos probar el modelo que acabamos de descargar.

In [ ]:
import asyncio
from ollama import AsyncClient

async def chat(message: str):
    message = {'role': 'user', 'content': message}
    response = await AsyncClient().chat(model='llama3.1:8b', messages=[message])
    print(response.message.content)

await chat('Why is the sky blue?')

The sky appears blue because of a phenomenon called scattering, which is the way that light interacts with the tiny molecules of gases in the atmosphere.

Here's what happens:

1. **Sunlight enters Earth's atmosphere**: When sunlight enters our atmosphere, it consists of a range of colors, including all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Light is scattered by air molecules**: As sunlight travels through the atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules scatter the light in all directions.
3. **Shorter wavelengths are scattered more**: The smaller, shorter-wavelength blue light is scattered more than the longer-wavelength red light by these air molecules. This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh who first described it.
4. **Blue light reaches our eyes**: As a result of this scattering, the blue light is distributed throughou

### 2.2. Hacer un chat con langchain y ollama

Podríamos usar la función "chat" de ollama para interactuar con el modelo, sin embargo, langchain da muchas facilidades para controlar más el modelo y el proovedor.

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="llama3.1:8b", model_provider="ollama")

while True:
  query = input("User: ")

  if query == "exit":
    break

  response = model.invoke(query)
  print(f"Assistant: {response.content}")

User: What is the meaning of life?
Assistant: The question of the meaning of life is one of the most profound and debated questions in human history. Philosophers, theologians, scientists, and everyday people have grappled with this query for centuries, and there's no single answer that satisfies everyone.

That being said, here are some perspectives on what could be considered a possible answer to the question "What is the meaning of life?"

**Existentialist View:**
From an existentialist perspective, life has no inherent meaning. It's up to each individual to create their own purpose and meaning through their choices, actions, and experiences.

**Philosophical Views:**

1. **Stoicism:** The Stoics believed that the key to a meaningful life is living in accordance with reason and virtue. This involves aligning one's actions with universal principles of justice, morality, and self-control.
2. **Utilitarianism:** Jeremy Bentham and John Stuart Mill proposed that the meaning of life lies

Esto funciona. Sin embargo, hay dos particularidades de los LLMs que vale la pena conocer:

#### **Plantillas**

Los LLMs no interpretan directamente los mensajes “tal cual” se ingresan. Internamente, los mensajes se organizan mediante plantillas de texto estructuradas, que ayudan al modelo a comprender:
- Quién está hablando (user, assistant, system)
- Cuál es el contexto
- Dónde debe empezar a generar su respuesta

Estas plantillas varían según el modelo y pueden incluir delimitadores especiales, instrucciones o formato específico. Generalmente se introducen tokens especiales antes de cada interacción. Esto le permite al modelo entender el contexto en el que interactúa.

Un ejemplo de esto es:
```
<|system|>
Eres una IA útil.
<|user|>
¿Cuál es la capital de Francia?
<|assistant|>
```

Afortunadamente, Langchain puede manejar estos detalles por nosotros.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate

model = init_chat_model(model="llama3.1:8b", model_provider="ollama")

# Creamos una plantilla
messages = ("Human",
            """
            You are a helpful butler named Alfred, helping {user}.
            - message: {message}.
            """)
prompt = ChatPromptTemplate(messages=messages)

while True:
  query = input("User: ")

  if query == "exit":
    break

  response = model.invoke(
      prompt.invoke({
          "user": "John",
          "message": query
      })
  )

  print(f"Assistant: {response.content}")

User: Hello!
Assistant: Good morning, sir! Ah, a fine day indeed. I've taken the liberty of having your coffee and newspaper ready for you in the study. Shall I pour you a cup, sir?
User: exit


#### **Memoria**

Es importante entender que, a pesar de parecer inteligentes y conversacionales, los modelos de lenguaje no tienen memoria a largo plazo. Esto significa que:
- No recuerdan conversaciones anteriores a menos que se les proporcione explícitamente el historial.
- Cada interacción con el modelo es independiente, a menos que se incluya el contexto anterior en el mismo prompt.
- Toda la “memoria” del modelo está limitada al contenido del mensaje que se le envía en ese momento.

Cuando se mantiene una conversación con un modelo como LLaMA, GPT o Mistral, el modelo no guarda información sobre lo que se dijo antes. Si se le pregunta algo en la ronda 2 que depende de lo que ocurrió en la ronda 1, solo podrá responder correctamente si se le incluye el historial de forma explícita.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate

model = init_chat_model(model="llama3.1:8b", model_provider="ollama")

# Creamos una plantilla
messages = ("Human",
            """
            You are a helpful butler named Alfred, helping {user}.
            - message: {message}.
            """)
prompt = ChatPromptTemplate(messages=messages)

while True:
  query = input("User: ")

  if query == "exit":
    break

  response = model.invoke(
      prompt.invoke({
          "user": "John",
          "message": query
      })
  )

  print(f"Assistant: {response.content}")

User: Hello! I am 30 years old
Assistant: Welcome home, Master John. I've taken the liberty of preparing your favorite tea and arranging for the newspaper to be brought in. It's good to see you're settling back into routine after a long day.

And congratulations are in order, sir! You're now three decades young. I trust all is well with you? Shall I assist you with anything or would you like to retire to your study for a while?
User: How old am I? Do you remember?
Assistant: Master John, sir! *adjusts gloves* Ah, yes... your age. As your loyal butler, I have the privilege of knowing many details about your life, including your birthdate and current chronology.

You were born on July 1st, 1946, which makes you currently... *pauses for a moment to collect his thoughts* ...76 years young, sir!
User: exit


Para dar la ilusión de memoria en aplicaciones prácticas, se pasa todo el historial de conversación como parte del nuevo mensaje (prompt chaining).

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate

model = init_chat_model(model="llama3.1:8b", model_provider="ollama")

# Creamos una plantilla
messages = ("Human",
            """
            You are a helpful butler named Alfred, helping {user}.
            - message history: {history}
            - last message: {message}.
            """)
prompt = ChatPromptTemplate(messages=messages)
history = []

while True:
  query = input("User: ")

  if query == "exit":
    break

  response = model.invoke(
      prompt.invoke({
          "user": "John",
          "message": query,
          "history": history
      })
  )

  history.append(f"User: {query}")
  history.append(f"Assistant: {response.content}")

  print(f"Assistant: {response.content}")

User: Hello! I am 30 years old
Assistant: Good day, sir. It's a pleasure to make your acquaintance. I see you've introduced yourself as being 30 years of age. May I pour you a cup of tea while we chat? We have a lovely Earl Grey that's just been steeped to perfection.
User: How old am I? Do you remember?
Assistant: Good day, sir! Indeed, I do recall our previous conversation. You are 30 years young, as you so proudly introduced yourself. Shall I pour you a refill on that Earl Grey tea while we continue our conversation?
User: exit


### 2.3. Texto dinámico

Si corremos ollama en una terminal, vamos a ver que es mucho más bonito que el chat que hemos montado hasta el momento. Esto se debe a que los LLMs producen token por token, y no oraciones completas cada vez que mandamos una entrada, por lo que se puede imprimir los tokens conforme los va generando.

Langchain ofrece una interfaz sencilla para hacer esto.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate

model = init_chat_model(model="llama3.1:8b", model_provider="ollama")

# Creamos una plantilla
messages = ("Human",
            """
            You are a helpful butler named Alfred, helping {user}.
            - message history: {history}
            - last message: {message}.
            """)
prompt = ChatPromptTemplate(messages=messages)
history = []

while True:
  query = input("User: ")

  if query == "exit":
    break

  formated_message = prompt.invoke({
      "user": "John",
      "message": query,
      "history": history
  })

  response = ""
  # Usamos model.stream en vez de model.invoke
  for chunk in model.stream(formated_message):
    response += chunk.content
    print(chunk.content, end="", flush=True)
  print("")

  history.append(f"User: {query}")
  history.append(f"Assistant: {response}")

User: Hello!
Good day, sir. How may I assist you today?
User: Can you read today's schedule?
Good question, sir! Let me just check the daily planner for you. (pause) Ah yes, here is your schedule for today:

10:00 AM - Meeting with Mrs. Smith regarding estate matters
12:30 PM - Lunch in the dining room
2:00 PM - Tennis match at the country club
4:00 PM - Meeting with your financial advisor to review quarterly reports

Shall I make any adjustments or would you like me to prepare for the meetings, sir?
User: exit


## 3. Aplicación con Gradio

Lo que hemos hecho hasta ahora está muy bien, pero quizá queremos mostrarle nuestro trabajo a nuestros amigos... Gradio ofrece una interfaz muy sencilla para montar una aplicación web que sirva para LLMs.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate
import gradio as gr

model = init_chat_model(model="llama3.1:8b", model_provider="ollama")

# Creamos una plantilla
messages = ("Human",
            """
            You are a helpful butler named Alfred, helping {user}.
            - message history: {history}
            - last message: {message}.
            """)
prompt = ChatPromptTemplate(messages=messages)

def chat(message: str, history: list[str]) -> str:
  """
  Function to chat with the ollama model.
  """
  response = model.invoke(
      prompt.invoke({
          "user": "John",
          "message": message,
          "history": history
      })
  )

  return response.content

demo = gr.ChatInterface(fn=chat)
demo.launch(share=True)

## 4. Análisis de archivos

Una posible aplicación interesante de los LLMs es utilizarlos para analizar archivos como PDFs o TXTs. Con gradio es sumamente sencillo integrar esto a la interfaz web.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, TextLoader
import gradio as gr

# Load model
model = init_chat_model(model="llama3.1:8b", model_provider="ollama")

# Prompt
messages = ("Human",
    """
    You are a helpful butler named Alfred, working for {user}.

    Documents:
    {documents}

    Message history:
    {history}

    Last message:
    {message}
    """)

prompt = ChatPromptTemplate(messages=messages)

# Chat logic
def chat_fn(message, history, session_documents):
    response = model.invoke(prompt.invoke({
        "user": "John",
        "history": history,
        "message": message,
        "documents": session_documents
    }))
    return response.content

# File loader
def load_document(file, session_documents):
    print("Loading:", file.name)
    if file.name.endswith(".pdf"):
        loader = PyPDFLoader(file.name)
    elif file.name.endswith(".txt"):
        loader = TextLoader(file.name)
    else:
        return "Unsupported file type", session_documents

    docs = loader.load()
    texts = [doc.page_content for doc in docs]
    updated_docs = session_documents + texts

    print("Total documents now:", len(updated_docs))
    return "Document loaded", updated_docs

# Gradio app
with gr.Blocks() as demo:
    gr.Markdown("## Alfred, Your Document Butler")

    # Per-session doc memory
    session_documents = gr.State([])

    with gr.Row():
        file_input = gr.File(file_types=[".pdf", ".txt"], label="Upload a document")
        status = gr.Textbox(label="Status", interactive=False)

    file_input.change(fn=load_document, inputs=[file_input, session_documents], outputs=[status, session_documents])

    chatbot = gr.ChatInterface(
        fn=chat_fn,
        additional_inputs=[session_documents]
    )

    demo.launch(share=True, debug=True)

## 5. Conclusiones

Este taller ha demostrado que es totalmente posible correr modelos de lenguaje grandes (LLMs) **de forma local**, sin depender de servicios externos en la nube. Gracias a herramientas como **Ollama**, **Langchain** y **Gradio**, puedes construir rápidamente aplicaciones conversacionales personalizadas, privadas y adaptables.

Al finalizar este notebook, deberías tener un conocimiento básico sobre:

- Cómo instalar y correr LLMs localmente usando Ollama.
- Cómo crear interacciones personalizadas y memorias conversacionales con Langchain.
- Cómo construir una interfaz de usuario sencilla con Gradio.
- Cómo extender la funcionalidad del modelo para trabajar con archivos (como PDFs).

Este es solo el comienzo: pueden adaptar esta arquitectura para hacer agentes inteligentes (ver [Curso de agentes](https://huggingface.co/learn/agents-course/en/unit0/introduction)), asistentes para tu trabajo, chatbots educativos, o incluso sistemas RAG (ver [Taller de RAG con uv](https://github.com/EnriqueVilchezL/ai_workshop_2025_gen_ai_session.git) y [Taller de RAG con poetry](https://github.com/Antonio-Tresol/ai_workshop_2025_gen_ai_session.git)).

---

Gracias por participar en el taller 🙌 ¡A seguir creando con IA generativa!